In [3]:
############################################### IMPORTS ####################################
############ ALGEBRA #################
import numpy as np  # linear algebra
import pandas as pd  # data processing
import os
import json

############### SKLEARN ###############
from sklearn.model_selection import train_test_split 

############## TENSORFLOW ############
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import regularizers,layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import TextVectorization,Conv1D, MaxPooling1D, LSTM, Bidirectional, Dense, BatchNormalization, Dropout

In [4]:
tf.__version__

'2.10.1'

In [5]:
def map_emotion(value):
    if value == 0:
        return 'sadness'
    elif value == 1:
        return 'joy'
    elif value == 2:
        return 'love'
    elif value == 3:
        return 'anger'
    elif value == 4:
        return 'fear'
    else:
        return 'Surprise'

In [7]:
######## DATA ############
FILE_PATH='emotion-dataset'
df_train=pd.read_csv(os.path.join(FILE_PATH,'training.csv'))#Train dataset
df_test=pd.read_csv(os.path.join(FILE_PATH,'test.csv')) #Test dataset
df_val=pd.read_csv(os.path.join(FILE_PATH,'validation.csv'))#Validation dataset.

In [8]:
df_train=pd.concat([df_train, df_test,df_val], axis=0)

In [9]:
############### PREPARING DATASET ###################
X = df_train[['text']]
y = df_train[['label']]
X_train, X_eval, y_train, y_eval = train_test_split(X, y, test_size=0.1, random_state=42)
#Multi Categoryical Transformations
y_train = tf.keras.utils.to_categorical(y_train)
y_eval = tf.keras.utils.to_categorical(y_eval)

In [10]:
#PREPROCESSING OF DATASET
###########################################################################################
from nltk.corpus import stopwords
stopwords_english = set(stopwords.words('english'))
my_stopwords = set(["http", "'s", "n't", "'m", "'re", "'ve"])
stopwords_english.update(my_stopwords)

def preprocess_review(text):
    text = tf.strings.lower(text)
    text = tf.strings.regex_replace(text, r'\d+', '')
    text = tf.strings.regex_replace(text, '[^\w\s]', '')
    
    tokens = tf.strings.split(text)
    #Different Operations
    clean_text = tf.strings.reduce_join(tokens, separator=' ', axis=-1)
    
    return clean_text
###########################################################################

#Custom TextVectorization Layer
max_len=round(sum([len(i.split()) for i in X['text']])/len(X))
vectorize_layer=TextVectorization(
    max_tokens=50000,
    standardize='lower_and_strip_punctuation',
    split='whitespace',
    ngrams=None,
    output_mode='int',
    output_sequence_length=max_len,
    pad_to_max_tokens=True
    
)
vectorize_layer.adapt(X)
words_vocab=vectorize_layer.get_vocabulary()

###############################################################################
#Custom Embedding layer
embed_layer=layers.Embedding(
    input_dim=len(words_vocab),
    output_dim=128,
    embeddings_initializer='uniform',
    input_length=max_len,
)

################################################################################

In [11]:
###############################################################################################
#Custom CallBack and Scheduler
early_stopping = tf.keras.callbacks.EarlyStopping(
    patience=15,
    min_delta=0.001,
    restore_best_weights=True,
)

lr_schedule = tf.keras.optimizers.schedules.InverseTimeDecay(
  0.001,
  decay_steps=X.shape[0]*20,
  decay_rate=1,
  staircase=False)


##################################################################################
# Custom MODEL ASSEMBLING
def tf_model(vectorize_layer, embed_layer):
    model = Sequential()
    
    model.add(layers.Input(shape=(1,), dtype=tf.string))
    model.add(tf.keras.layers.Lambda(preprocess_review))
    model.add(vectorize_layer)
    model.add(embed_layer)
    
    model.add(layers.Conv1D(filters=64, kernel_size=3, activation='relu'))
    model.add(layers.MaxPooling1D(pool_size=2))

    model.add(layers.Bidirectional(layers.LSTM(units=64, return_sequences=True)))
    model.add(layers.BatchNormalization())
    model.add(layers.Dropout(0.2))

    model.add(layers.Bidirectional(layers.LSTM(units=128)))
    model.add(layers.BatchNormalization())
    model.add(layers.Dropout(0.2))

    model.add(layers.Dense(units=32, activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.Dropout(0.2))

    model.add(layers.Dense(units=6, activation='softmax'))
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=lr_schedule),
        loss='categorical_crossentropy',
        metrics=['acc']
    )

    return model

In [12]:
#Model fitting.
model=tf_model(vectorize_layer, embed_layer)
model.fit(X_train,y_train,validation_data=[X_eval,y_eval],epochs=3,
                callbacks=[early_stopping],batch_size=50)

Epoch 1/30
360/360 [==============================] - 76s 176ms/step - loss: 1.2099 - acc: 0.5591 - val_loss: 1.1261 - val_acc: 0.6660
Epoch 2/30
360/360 [==============================] - 64s 179ms/step - loss: 0.4214 - acc: 0.8579 - val_loss: 0.5003 - val_acc: 0.8270
Epoch 3/30
360/360 [==============================] - 53s 146ms/step - loss: 0.2513 - acc: 0.9186 - val_loss: 0.5340 - val_acc: 0.8185
Epoch 4/30
360/360 [==============================] - 59s 163ms/step - loss: 0.1640 - acc: 0.9484 - val_loss: 0.7851 - val_acc: 0.7830
Epoch 5/30
360/360 [==============================] - 43s 119ms/step - loss: 0.1339 - acc: 0.9581 - val_loss: 0.7394 - val_acc: 0.8100
Epoch 6/30
360/360 [==============================] - 60s 166ms/step - loss: 0.1011 - acc: 0.9681 - val_loss: 0.7962 - val_acc: 0.8130
Epoch 7/30
360/360 [==============================] - 60s 167ms/step - loss: 0.0912 - acc: 0.9712 - val_loss: 0.7248 - val_acc: 0.8250
Epoch 8/30
360/360 [==============================] - 5

In [14]:
Result=np.argpartition(-model.predict(['I am happy about my life']), kth=2, axis=1)[:, :2]
[[map_emotion(value) for value in row] for row in Result]

1/1 [==============================] - 3s 3s/step


[['joy', 'fear']]

In [19]:
from keras.models import load_model
model.save('model.tf')

INFO:tensorflow:Assets written to: model.tf\assets


INFO:tensorflow:Assets written to: model.tf\assets
